In [ ]:
import pandas as pd
import requests
import plotly.graph_objects as go
import numpy as np
import GMX_data as gmx

In [ ]:
symbol = gmx.get_token()
actions = gmx.get_actions()
end_day = '2023061200'
glp = gmx.get_glp(end_day)
fee = gmx.get_fee(end_day)
prc = gmx.get_fast_price(symbol,end_day)
result,fig = gmx.show_case(glp,fee,prc)

KeyboardInterrupt: 

In [ ]:
fig.show()